Here we try to simulate proteoform mixtures as discussed during the Lorentz center conference on Computational Proteomics.

A simulations only including 2 proteoforms.  
First some constants. 

In [1]:
number_of_replicates = 3
number_of_conditions = 2
number_of_shared_peptides = 4
number_of_uniq_peptides_per_proteoform = 4
number_of_peptides = number_of_uniq_peptides_per_proteoform*2 + number_of_shared_peptides

Also we define the proteoform concentrations for each condition

In [2]:
prot1_per_condition = [1.0, 1.0] # A, B
prot2_per_condition = [0.3, 3.0]

Each peptide will have a ionization efficency that we assume is log-normal distributed.

In [3]:
ionization_mu, ionization_sigma = 3., 5.

Here we introduce a noise model, which adds normal distributed noise with a standard deviation proportional to the signal, but with a minimal stdv.

In [4]:
import numpy as np

noise_proportionality = 0.2
noise_min = 0.1

rng = np.random.default_rng()

def add_noise(raw):
    stdv = max(noise_min, raw * noise_proportionality)
    noise = rng.normal(0., stdv)
    return abs(raw + noise) # guard ourseves from negative values
    
add_noise_v = np.vectorize(add_noise)

Given these constraints we can now simulate peptide concentrations.

In [5]:
peptide_conc_A = np.asarray([prot1_per_condition[0]]*number_of_uniq_peptides_per_proteoform + \
                 [prot1_per_condition[0]+prot2_per_condition[0]]*number_of_shared_peptides + \
                 [prot2_per_condition[0]]*number_of_uniq_peptides_per_proteoform)

peptide_conc_B = np.asarray([prot1_per_condition[1]]*number_of_uniq_peptides_per_proteoform + \
                 [prot1_per_condition[1]+prot2_per_condition[1]]*number_of_shared_peptides + \
                 [prot2_per_condition[1]]*number_of_uniq_peptides_per_proteoform)

peptide_conc = np.vstack((np.tile(peptide_conc_A, (number_of_replicates, 1)),\
                          np.tile(peptide_conc_B, (number_of_replicates, 1))))


And then we apply ionization efficency and noise

In [6]:
ionization = rng.lognormal(ionization_mu, ionization_sigma, number_of_peptides)
peptide_abundance_raw = np.multiply(ionization,peptide_conc)
peptide_abundance = add_noise_v(peptide_abundance_raw)

## An idea of how to decompose convolved proteoform abindances 
Given our convolved peptide messurments, could we detect that there are two different proteofoms present?

In [7]:
from numpy.linalg import svd
from scipy import stats
x=np.log(peptide_abundance.T)
#x=(peptide_abundance.T)
y = x - x.mean(axis=0, keepdims=True) #subtract row mean 

U, S, Vt = svd(y,full_matrices=False)
eigen_proteins = (Vt.T)[:,0:2]
print(f"First component p=%f"%stats.ttest_ind(eigen_proteins[0:3,0],eigen_proteins[3:6,0], equal_var = False)[1])
print(f"Second component p=%f"%stats.ttest_ind(eigen_proteins[0:3,1],eigen_proteins[3:6,1], equal_var = False)[1])


First component p=0.001778
Second component p=0.000642


In [8]:
eigen_proteins

array([[ 0.38270997, -0.3370669 ],
       [ 0.38001612, -0.44706525],
       [ 0.36309108, -0.51831023],
       [ 0.43171577,  0.4325003 ],
       [ 0.45042034,  0.34311498],
       [ 0.43362276,  0.33628659]])